# Install Dependencies

In [123]:
!python -m pip install beautifulsoup4 yfinance

# Import Components

In [124]:
%run ./../../components/data_access/html_data_access.py

In [125]:
%run ./../../components/data_access/easy_equities_data_access.py

In [126]:
%run ./../../components/data_access/yahoo_finance_data_access.py

In [127]:
%run ./../../components/engines/valuation_engine.py

In [128]:
html_data_access = HtmlDataAccess()
easy_equities_data_acsess = EasyEquitiesDataAccess()
yahoo_finance_data_access = YahooFinanceDataAccess()
valuation_engine = ValuationEngine(html_data_access=html_data_access)

In [129]:
vars(valuation_engine.valuate(company_name='Test', symbol='AAPL'))

EPS: 6.05, EPS Next 5 Years: 0.0983%
P/E Ratio: 26.42, Current Price: $ 164.87
Intrinsic Value: $ 122.95461538461537 vs. Current Price: $ 164.87


{'symbol': 'AAPL',
 'company_name': 'Test',
 'current_price': 164.87,
 'valuation_price': 122.95461538461537,
 'absolute_current_v_valuation_delta': 0.25423293877227293,
 'is_overvalued': True,
 'annual_dividend_percentage': 0.56,
 'purchased_price': None}

# Import Dependencies

In [130]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import pandas as pd
from datetime import datetime, timedelta

# Implementation

In [131]:
def get_plot_for_symbol(symbol: str,
                        symbol_data: pd.DataFrame,
                        valuation: ValuationResult,
                        transactions: list = list(), 
                        fontsize: int = 7) -> object:
    
    if len(transactions) > 0:
        transactions_by_date = sorted(transactions, key=lambda i: i['date'])
        symbol_data = symbol_data.loc[transactions_by_date[0]['date']:]
        valuation.purchased_price = symbol_data.Close.values[0]
    
    with plt.rc_context({
            'lines.linewidth': 0.75,
            'figure.figsize': (6, 3)
        }):
        data_to_visualize = symbol_data.Close
        fig, axs = plt.subplots(1, dpi=600, layout='constrained')
        plot = data_to_visualize.plot(ax=axs, color='r' if symbol_data.iloc[0].Close > symbol_data.iloc[-1].Close else 'g', fontsize=fontsize)
        mid = (fig.subplotpars.right + fig.subplotpars.left)/2
        fig.suptitle(f'{symbol}', fontsize=fontsize+2, x=mid)
        
        if valuation.annual_dividend_percentage is not None:
            #axs[0]
            axs.set_title(f'Annual Dividend Yield: {round(valuation.annual_dividend_percentage, 2)}%', fontdict={'fontsize': fontsize, 'fontweight': 'light'})
        
        plot.set_xlabel('')
        plot.set_ylabel('')
        
        for transaction in sorted(transactions, key=lambda i: i['date']):
            plot.axvline(transaction['date'], linestyle='-.', color=('r' if symbol_data._get_value(transaction['date'], 'Close') > valuation.current_price else 'g'))
        
        tick = mtick.StrMethodFormatter('${x:,.0f}')
        valuation_delta: float = round(valuation.absolute_current_v_valuation_delta * 100, 2)
        
        plot.yaxis.set_major_formatter(tick)
        # Valuation price.
        plot.axhline(valuation.valuation_price, color='r' if valuation.valuation_price < symbol_data.iloc[-1].Close else 'g', linestyle='--', label=f'Valuation (${round(valuation.valuation_price, 2)} | Δ{valuation_delta}%)')
        
        # t-1day
        plot.axvline(datetime.now() - timedelta(days=1), linestyle='-.', color='y')
        # t-7days
        if symbol_data.shape[0] > 7:
            plot.axvline(datetime.now() - timedelta(days=7), linestyle='-.', color='y')
        # t-30days
        if symbol_data.shape[0] > 30:
            plot.axvline(datetime.now() - timedelta(days=30), linestyle='-.', color='y')
        # t-6months
        if symbol_data.shape[0] > 30*6:
            plot.axvline(datetime.now() - timedelta(days=30*6), linestyle='-.', color='y')
        # t-1year
        if symbol_data.shape[0] > 365:
            plot.axvline(datetime.now() - timedelta(days=365), linestyle='-.', color='y')
        # t-5years
        if symbol_data.shape[0] > (365*5):
            plot.axvline(datetime.now() - timedelta(days=365*5), linestyle='-.', color='y')
        # t-10years
        if symbol_data.shape[0] > 365*10:
            plot.axvline(datetime.now() - timedelta(days=365*10), linestyle='-.', color='y')
        
        handles, labels = plot.get_legend_handles_labels()
        first_value: float = symbol_data.iloc[0].Close
        current_price: float = symbol_data.iloc[-1].Close
        delta_since_purchase: float = round(100 - min(current_price, first_value) / max(current_price, first_value) * 100, 2)
        labels[0] = f'{labels[0]} (${round(current_price, 2)} | Δ{delta_since_purchase}%)'
        plot.legend(handles=handles[:], labels=labels[:], fontsize=fontsize)

        return plot

# Usage

In [132]:

owned_plots = list()
should_render_owned_stocks = True

if should_render_owned_stocks:
    personal_orders = easy_equities_data_acsess.get_personal_transactions()

    for symbol in personal_orders:
        valuation = valuation_engine.valuate(company_name=personal_orders[symbol]['name'], symbol=symbol)
        hist = yahoo_finance_data_access.get_symbol_history(symbol=symbol)
        owned_plots.append((get_plot_for_symbol(symbol=f'{valuation.company_name} | {valuation.symbol}',
                                symbol_data=hist,
                                valuation=valuation,
                                transactions=personal_orders[symbol]['transactions']), valuation))

    owned_plots = sorted(owned_plots, key=lambda i: i[1].current_price, reverse=True) 

available_plots = list()
should_render_available_stocks = True

if should_render_available_stocks:
    available_us_stock = easy_equities_data_acsess.get_supported_assets()
    
    for name, symbol in available_us_stock:
        try:
            valuation = valuation_engine.valuate(company_name=name, symbol=symbol)
            
            if valuation.is_overvalued:
                continue
            
            ticker = yf.Ticker(symbol)
            hist = yahoo_finance_data_access.get_symbol_history(symbol=symbol)
            available_plots.append((get_plot_for_symbol(symbol=f'{valuation.company_name} | {valuation.symbol}',
                                    symbol_data=hist,
                                    valuation=valuation), valuation))
        except Exception:
            pass

    available_plots = sorted(available_plots, key=lambda i: i[1].current_price, reverse=True) 

EPS: 8.33, EPS Next 5 Years: 0.5189%
P/E Ratio: 77.905, Current Price: $ 871.27
Intrinsic Value: $ 499.19126923076914 vs. Current Price: $ 871.27
EPS: 6.05, EPS Next 5 Years: 0.0983%
P/E Ratio: 26.42, Current Price: $ 164.87
Intrinsic Value: $ 122.95461538461537 vs. Current Price: $ 164.87
EPS: 2.0, EPS Next 5 Years: 0.3%
P/E Ratio: 38.714999999999996, Current Price: $ 13.24
Intrinsic Value: $ 59.56153846153846 vs. Current Price: $ 13.24
EPS: 2.69, EPS Next 5 Years: 0.0403%
P/E Ratio: 8.945, Current Price: $ 10.6
Intrinsic Value: $ 18.50926923076923 vs. Current Price: $ 10.6
EPS: 2.0, EPS Next 5 Years: 0.3%
P/E Ratio: 38.714999999999996, Current Price: $ 13.24
Intrinsic Value: $ 59.56153846153846 vs. Current Price: $ 13.24
EPS: 7.15, EPS Next 5 Years: 0.004%
P/E Ratio: 17.215, Current Price: $ 148.48
Intrinsic Value: $ 94.68250000000002 vs. Current Price: $ 148.48
EPS: 1.12, EPS Next 5 Years: 0.12%
P/E Ratio: 41.04, Current Price: $ 61.78
Intrinsic Value: $ 35.35753846153846 vs. Curren